# Homodyne tomography

### What is Homodyne tomography?

Homodyne tomography is a method for reconstructing quantum states of light by measuring field quadratures $Q_\theta$ at different local oscillator (LO) phases $\theta$ using a Homodyne detection scheme. From these measurements, one can recover the Wigner function $W(Q,P)$ and density matrix $\rho$, which makes it a key tool for verifying the preparation of nonclassical states in quantum optics.

<img src="../images/HomodyneTomography.png" alt="Homodyne Tomography" height="250"/>
<img src="../images/HomodyneDetector.png" alt="Homodyne Detector" height="250"/>

In the paper *Optical Continuous-Variable Qubit* (see references), superpositions of squeezed vacuum and squeezed single-photon states were engineered using photon subtraction. The setup employed an optical parametric oscillator (OPO) to generate squeezed vacuum, a small fraction of which was tapped off, displaced, and detected by an avalanche photodiode (APD). Conditional detection prepared the desired superpositions, which were then analyzed using balanced homodyne detection across multiple LO phases.

<img src="../images/ExperimentalSetup.png" alt="Homodyne Tomography" height="250"/>

In this notebook, we work with the experimental data obtained using this setup. Our goal is to benchmark and compare two different reconstruction methods - Maximum Likelihood Estimation and Bayesian Mean Estimation - to reproduce the Wigner functions reported in the paper.

### Loading and processing data

### Calculating quadrature values

### Reconstruction algoritms

#### Itterative Maxium Likelihood Estimation (iMLE)

#### Bayesian Mean Estimation (BME)

### Benchmarks and comparisons

In [1]:
# Importing libraries
import numpy as np
import scipy as sp
import pandas as pd
import os
from pathlib import Path
import sys
import json
from scipy.fft import rfft, irfft, rfftfreq
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

# Importing functions


# Global plot style settings
plt.rcParams.update({
    "font.family": "serif",
    "font.serif": ["DejaVu Serif"],
    "mathtext.fontset": "cm",            # Use Computer Modern for math
    "font.size": 14,                     # Base font size
    "axes.labelsize": 16,                # Axis label font size
    "axes.titlesize": 16,                # Title font size
    "legend.fontsize": 13,               # Legend font size
    "xtick.labelsize": 13,               # X tick label size
    "ytick.labelsize": 13,               # Y tick label size
    "axes.linewidth": 1.2,               # Thicker axis lines
    "xtick.direction": "in",             # x-yick direction
    "ytick.direction": "in",             # y-tick direction
    "text.usetex": False,                # Enable LaTeX if needed
    "figure.dpi": 100,                   # Good resolution for screens
    "savefig.dpi": 300                   # High resolution for saving
})

# Define function for nice plotting
def PlotSettings(ax, gridlines=False, minimalist=False):
    # Minor ticks
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    # Minimalist style
    if minimalist:
        # Hide top and right spines (borders)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        # Ticks only on bottom and left
        ax.tick_params(which='both', top=False, right=False)
    else:
        # Tick parameters
        ax.tick_params(which='both', direction='in', top=True, right=True)
        ax.tick_params(which='major', length=7, width=1.2)
        ax.tick_params(which='minor', length=4, width=1)
    # Optional grid
    if gridlines:
        ax.grid(True, which="major", linestyle="--", linewidth=0.6, alpha=0.7)
        ax.grid(True, which="minor", linestyle=":", linewidth=0.5, alpha=0.4)

In [6]:
import tkinter as tk
from tkinter import filedialog, messagebox
import os

# Files we expect in the folder
REQUIRED_FILES = ["data.txt", "config.json"]

folder_path = None

def select_folder():
    global folder_path
    folder = filedialog.askdirectory(title="Select a folder", parent=root)
    if folder:
        folder_path = folder
        label.config(text=f"Selected: {folder}")
        print("Selected folder:", folder)
    else:
        folder_path = None
        label.config(text="No folder selected")

def process_data():
    if not folder_path:
        messagebox.showerror("Error", "No folder selected!")
        return

    # Check required files
    missing = [f for f in REQUIRED_FILES if not os.path.isfile(os.path.join(folder_path, f))]
    if missing:
        messagebox.showerror("Error", f"Missing required files: {', '.join(missing)}")
        return

    # If everything is fine
    messagebox.showinfo("Success", "Folder contains all required files. Processing...")
    print("Processing data in:", folder_path)
    # ---- Your data processing code goes here ----

root = tk.Tk()
root.title("Folder Selector")

# Make sure the window starts in front
root.lift()
root.attributes('-topmost', True)
root.after_idle(root.attributes, '-topmost', False)

# Buttons
button_select = tk.Button(root, text="Choose Folder", command=select_folder, padx=10, pady=5)
button_select.pack(pady=10)

button_process = tk.Button(root, text="Process Data", command=process_data, padx=10, pady=5)
button_process.pack(pady=10)

# Label
label = tk.Label(root, text="No folder selected", wraplength=400)
label.pack(pady=10)

root.mainloop()


Selected folder: C:/Users/tbrav/OneDrive - Danmarks Tekniske Universitet/Engineering Physics - MSc/Sem 2/10387 Scientific Computing in Quantum Information Science/Homodyne-Tomograpy/data/data-tora


KeyboardInterrupt: 